In [10]:
import nltk
import string
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import os
import numpy as np
import math
import heapq
import json

In [11]:
data = ""
tokens = []
corpus = []
wordfreq = {}
word_idf_values = {}
word_tf_values = {}
tfidf_values = []
tfidf = {}
current_path = os.getcwd()

In [12]:
def DataRetrival():
    global data, tokens, corpus, wordfreq, word_idf_values, word_tf_values, tfidf_values, tfidf, current_path
    ans = []
    for file_name in os.listdir(current_path):
        if file_name.endswith('txt'):
            file = open(file_name, 'r')
            ans.append(file.read())
            file.close()
    data = "".join(i for i in ans)
    print("Data Retrieved")

def RemovePunct():
    global data, tokens, corpus, wordfreq, word_idf_values, word_tf_values, tfidf_values, tfidf, current_path
    data = "".join([char for char in data if char not in string.punctuation])
    print()
    print("Punctuations Removed")

def Tokenization():
    global data, tokens, corpus, wordfreq, word_idf_values, word_tf_values, tfidf_values, tfidf, current_path
    tokens = word_tokenize(data)
    print()
    print("Tokens Made")
    
def RemoveStopWords():
    global data, tokens, corpus, wordfreq, word_idf_values, word_tf_values, tfidf_values, tfidf, current_path
    tokens = [word for word in tokens if not word in stopwords.words()]
    print()
    print("Stop words removed")
    
def Stemming():
    global data, tokens, corpus, wordfreq, word_idf_values, word_tf_values, tfidf_values, tfidf, current_path
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    print()
    print("Stemming done")
    
def FrequencyCounting():
    global data, tokens, corpus, wordfreq, word_idf_values, word_tf_values, tfidf_values, tfidf, current_path
    for file_name in os.listdir(current_path):
        if file_name.endswith('txt'):
            print(file_name)
            file = open(file_name,'r')
            temp_data = file.read()
            temp_tokens = nltk.word_tokenize(temp_data)
            temp_tokens = [word for word in temp_tokens if not word in nltk.corpus.stopwords.words()]
            stemmer = PorterStemmer()
            temp_tokens = [stemmer.stem(word) for word in temp_tokens]
            for temp_token in temp_tokens:
                if temp_token in tokens and temp_token not in wordfreq.keys():
                    wordfreq[temp_token] = 1
                elif temp_token in tokens:
                    wordfreq[temp_token] += 1
            file.close()
    print()
    print("Frequency Counted")
    
def TF_and_IDF():
    global data, tokens, corpus, wordfreq, word_idf_values, word_tf_values, tfidf_values, tfidf, current_path
    for word in wordfreq:
        count = 0
        tf_vector=[]
        for file_name in os.listdir(current_path):
            freq = 0
            if file_name.endswith('txt'):
                file = open(file_name, 'r')
                temp_data = file.read()
                temp_tokens = nltk.word_tokenize(temp_data)
                temp_tokens = [word for word in temp_tokens if not word in nltk.corpus.stopwords.words()]
                stemmer = PorterStemmer()
                temp_tokens = [stemmer.stem(word) for word in temp_tokens]
                if word in temp_tokens:
                    count += 1
                for w in temp_tokens:
                    if word == w:
                        freq += 1
                word_tf = freq / len(temp_tokens)
                tf_vector.append(word_tf)
                file.close()
        word_idf_values[word] = np.log(10 / (1 + count))
        word_tf_values[word] = tf_vector
    print()
    print("TF and IDF Calculated")
    
# def TF():
#     global data, tokens, corpus, wordfreq, word_idf_values, word_tf_values, tfidf_values, tfidf, current_path
#     for word in wordfreq:
#         tf_vector=[]
#         for file_name in os.listdir(current_path):
#             if file_name.endswith('txt'):
#                 freq = 0
#                 file = open(file_name, 'r')
#                 temp_data = file.read()
#                 temp_tokens = nltk.word_tokenize(temp_data)
#                 temp_tokens = [word for word in temp_tokens if not word in nltk.corpus.stopwords.words()]
#                 stemmer = PorterStemmer()
#                 temp_tokens = [stemmer.stem(word) for word in temp_tokens]
#                 for w in temp_tokens:
#                     if word == w:
#                         freq += 1
#                 word_tf = freq / len(temp_tokens)
#                 tf_vector.append(word_tf)
#                 file.close()
#         word_tf_values[word] = tf_vector
#     print()
#     print("TF Calculated")

def TF_IDF():
    global data, tokens, corpus, wordfreq, word_idf_values, word_tf_values, tfidf_values, tfidf, current_path
    i = 0
    for word in word_tf_values.keys():
        temp_tfidf=[]
        for sen in word_tf_values[word]:
            score = sen * word_idf_values[word]
            temp_tfidf.append(score)
        tfidf_values.append(temp_tfidf)
        tfidf[word] = i
        i += 1
    print()
    print("TF-IDF Calculated")

def preprocessing():
    json_data = {}
    
    # Reading the data
    DataRetrival()
    
    data.lower()
    print()
    print("Data Lowercased")
    
    # Removing punctuations.
    RemovePunct()

    # Doing tokenization
    Tokenization()

    # Stop word removal
    RemoveStopWords()
    
    # Stemming
    Stemming()

    # Building Corpus
    corpus = tokens
    print()
    print("Corpus ready")

    # Counting frequancy of words in tokens
    FrequencyCounting()
                    
    # Calculating TF and IDF values
    TF_and_IDF()

    # Calculating tf values
    # TF()    
    
    # Calculating tf-idf value
    TF_IDF()
    
    json_data['preprocessing'] = {}
    json_data['preprocessing']['data'] = data
    json_data['preprocessing']['tokens'] = tokens
    json_data['preprocessing']['corpus'] = corpus
    json_data['preprocessing']['wordfreq'] = wordfreq
    json_data['preprocessing']['word_idf_values'] = word_idf_values
    json_data['preprocessing']['word_tf_values'] = word_tf_values
    json_data['preprocessing']['tfidf_values'] = tfidf_values
    json_data['preprocessing']['tfidf'] = tfidf
    with open('json_data.json', 'w') as outfile:
        json.dump(json_data, outfile)
    
    print()
    print("Preprocessing Done")
    print()
    print(json_data)

In [13]:
preprocessing()

Data Retieved

Data Lowercased

Punctuations Removed

Tokens Made

Stop words removed

Stemming done

Corpus ready
the_rolling_stones.txt
kings_of_leon.txt
coldplay.txt
imagine_dragons.txt
the_beatles.txt
one_republic.txt
green_day.txt
nirvana.txt
linkin_park.txt
maroon_5.txt

Frequency Counted


KeyboardInterrupt: 

In [7]:
from tkinter import *
from tkinter import ttk

In [8]:
def search():
    canvas.delete("all")
    infile = open("json_data.json", 'r')
    json_data = json.load(infile)
    infile.close()
    data = json_data['preprocessing']['data']
    tokens = json_data['preprocessing']['tokens']
    corpus = json_data['preprocessing']['corpus']
    wordfreq = json_data['preprocessing']['wordfreq']
    word_idf_values = json_data['preprocessing']['word_idf_values']
    word_tf_values = json_data['preprocessing']['word_tf_values']
    tfidf_values = json_data['preprocessing']['tfidf_values']
    tfidf = json_data['preprocessing']['tfidf']
    
    tf_idf_model = np.asarray(tfidf_values)
    tf_idf_model = np.transpose(tf_idf_model)
    det = []
    for row in tf_idf_model:
        sum = 0
        for num in row:
            sum += num ** 2
        sum = math.sqrt(sum)
        det.append(sum)
    s = searchEntry.get()
    s.lower()
    s = nltk.word_tokenize(s)
    s = [word for word in s if not word in nltk.corpus.stopwords.words()]
    stemmer = PorterStemmer()
    s = [stemmer.stem(word) for word in s]
    l = len(s)
    arr = []
    dic = {}
    for i in range(tf_idf_model.shape[0]):
        sum = 0.0
        for word in s:
            index = 0
            if word in tfidf.keys():
                index = tfidf[word]
                sum += tf_idf_model[i][index]
        arr.append(sum / (det[i] * math.sqrt(l)))
        dic[i] = sum / (det[i] * math.sqrt(l))
    heapq.heapify(arr)
    ans=[]
    ans=heapq.nlargest(10,arr)
    print(ans)
    print_text = ""
    for i in ans:
        id = 0
        for ind, val in dic.items():
            if val == i:
                id = ind + 1
                break
        print_text += ("Document: " + str(id) + "\n")
    canvas.create_text(40, 90, anchor=W, font="Ubuntu", text="Search Results: \n\n" + print_text)

In [9]:
root = Tk()
root.title("Harsh's Search Engine")
root.maxsize(900,600)
root.config(bg="white")

UI_frame = Frame(root, width=600, height=200, bg='white')
UI_frame.grid(row=0, column=0, padx=10, pady=5)

canvas = Canvas(root, width=1200, height=760, bg="ghost white")
canvas.grid(row=1, column=0, padx=10, pady=5)

searchEntry = Entry(UI_frame)
searchEntry.grid(row=1, column=1, padx=5, pady=5, sticky=W)
Button(UI_frame, text='Click to search',command = search, bg='coral').grid(row=1,column=2,padx=5,pady=5)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-8-a1b162080354>", line 3, in search
    infile = open("json_data.json", 'r')
FileNotFoundError: [Errno 2] No such file or directory: 'json_data.json'
